In [23]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('all_task_environments.csv', header=0, skiprows=[1])

In [36]:
objects = ['t', 'n', 's', 'p', 'c']

coordinates = {'c': (1,4,4),
              'n': (1,1,2),
              'p': (1,4,4),
              's': (1,2,2),
              't': (1,1,2),
              'start': (2,1,2),
              'table': (4,3,2)}

c1 = {'c': 1.5,
    'n': 1.0,
    'p': 1.5,
    's': 1.5,
    't': 1.0}

k1 = {'c': 1.0,
    'n': 0.4,
    'p': 0.4,
    's': 1.0,
    't': 0.3}

start_coordinates = [[2,1.5,2],[4,3,2],[4,3,2],[4,3,2],[4,3,2]]

In [4]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,], steps=1):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        #print(minval)
        del possible_items[minval]
        
        if steps == 2 and bool(possible_items) == True:
            for obj in possible_items.keys():
                possible_items[obj] = ((distance.euclidean(
                                    new_coords[minval],
                                    new_coords[obj])
                                    )
                                    ** k[obj]) * c[obj]
            #print(possible_items)
            minval_step2 = min(possible_items.values())
            minval_step2 = [k for k, v in possible_items.items() if v == minval_step2]
            minval_step2 = random.choice(minval_step2) # choose prediction randomly if multiple items have same cost
            #print(minval_step2 + '____')
            del possible_items[minval_step2]
            prediction.append(''.join([minval, minval_step2]))
            coord_index += 1
            
        else:
            prediction.append(minval)
          
        coord_index += 1
        
    return prediction

In [38]:
predict_sequence(objects, coordinates, start_coordinates, c1, k1, steps=2)

['tn', 'pc', 's']

In [12]:
def get_average(objects, coordinates, start_coordinates, c, k, dimension, sequence, steps=1):

    edit_list = []

    for x in range(0,100):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    avg = np.mean(edit_list)
    return avg

In [13]:
def get_avg_editdist(data, steps=1):
    results = pd.DataFrame()
    dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]
        
    for row in range(0,len(df)):
        objects = list(df.at[row,'objects'].split(','))
        strong_k = list(df.at[row,'strong_k'].split(','))
        mid_k = list(df.at[row,'mid_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
        start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
        sequence = str(df.at[row,'sequence'])
        
        #for k in np.arange(0.0,1.0,0.1):
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else 1.0 for obj in objects}
                    
            for c in np.arange(1.0,2.0,0.1):
                c = round(c, 1)
                c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
                
                for dim in dimensions:                
                    # get average edit distance
                    edit_dist = get_average(objects, coordinates, start_coordinates, c1, k1, dim, sequence)
                    edit_dist = edit_dist / len(sequence)
                    
                    #params = 'c: ' + str(c) + ', k: ' + str(k_strong) + ',' + str(k_mid) + ', dim: ' + str(dim[1])
                    params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                    results.at[row,params] = edit_dist
                    
    return results

#TODO:
# range for diff between strong/mid k?
# range for c and k in general?

In [14]:
#%%timeit -n1 -r1
# generate data for 2 steps of planning
results_new = get_avg_editdist(df, steps=2)

In [15]:
results_new

,"c: 1.0; k: 0.0,0.1; x","c: 1.0; k: 0.0,0.1; y","c: 1.0; k: 0.0,0.1; z","c: 1.0; k: 0.0,0.1; xy","c: 1.0; k: 0.0,0.1; xz","c: 1.0; k: 0.0,0.1; yz","c: 1.0; k: 0.0,0.1; xyz","c: 1.1; k: 0.0,0.1; x","c: 1.1; k: 0.0,0.1; y","c: 1.1; k: 0.0,0.1; z",...,"c: 1.8; k: 0.8,0.9; xz","c: 1.8; k: 0.8,0.9; yz","c: 1.8; k: 0.8,0.9; xyz","c: 1.9; k: 0.8,0.9; x","c: 1.9; k: 0.8,0.9; y","c: 1.9; k: 0.8,0.9; z","c: 1.9; k: 0.8,0.9; xy","c: 1.9; k: 0.8,0.9; xz","c: 1.9; k: 0.8,0.9; yz","c: 1.9; k: 0.8,0.9; xyz"
0,0.382000,0.862000,0.704000,0.288000,0.238000,0.400000,0.000000,0.196000,0.604000,0.694000,...,0.302000,0.400000,0.200000,0.198000,0.470000,0.474000,0.100000,0.298000,0.400000,0.200000
1,0.394000,0.600000,0.700000,0.200000,0.330000,0.600000,0.200000,0.190000,0.600000,0.712000,...,0.506000,0.600000,0.400000,0.204000,0.600000,0.548000,0.400000,0.502000,0.600000,0.400000
2,0.438000,0.600000,0.700000,0.000000,0.210000,0.600000,0.000000,0.220000,0.600000,0.708000,...,0.300000,0.400000,0.200000,0.212000,0.400000,0.472000,0.200000,0.306000,0.400000,0.200000
3,0.554000,0.400000,0.400000,0.200000,0.320000,0.400000,0.200000,0.416000,0.400000,0.400000,...,0.086000,0.200000,0.000000,0.420000,0.200000,0.250000,0.000000,0.094000,0.200000,0.000000
4,0.380000,0.666667,0.666667,0.506667,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,...,0.000000,0.480000,0.333333,0.000000,0.510000,0.153333,0.490000,0.000000,0.513333,0.333333
5,0.696000,0.516000,0.800000,0.596000,0.400000,0.310000,0.400000,0.708000,0.280000,0.800000,...,0.400000,0.800000,0.800000,0.684000,0.400000,0.800000,0.596000,0.400000,0.800000,0.800000
6,0.805000,0.710000,0.765000,1.000000,0.755000,1.000000,1.000000,0.745000,0.795000,0.695000,...,0.725000,0.500000,0.500000,0.755000,0.500000,0.735000,0.500000,0.710000,0.500000,0.500000
7,0.745000,1.000000,0.740000,1.000000,0.735000,1.000000,1.000000,1.000000,1.000000,0.732500,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.687500,1.000000,1.000000,1.000000,1.000000
8,0.163333,0.353333,0.443333,0.000000,0.176667,0.326667,0.000000,0.140000,0.340000,0.436667,...,0.146667,0.340000,0.000000,0.173333,0.333333,0.466667,0.000000,0.160000,0.273333,0.000000
9,0.176667,0.483333,0.186667,0.333333,0.153333,0.333333,0.333333,0.160000,0.500000,0.163333,...,0.173333,0.333333,0.333333,0.163333,0.493333,0.150000,0.333333,0.180000,0.333333,0.333333


In [20]:
# generate data for 1 step of planning
results1 = get_avg_editdist(df, steps=1)

In [16]:
def get_lowest_error(results):
    for col in list(results):
        results.loc['mean',col] = results[col].mean()
    lowest = min(results.loc['mean'])
    mean = sorted(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [25]:
# results for 2 steps
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

(0.4155257936507936, Index(['c: 1.5; k: 0.3,0.4; xy'], dtype='object'))

In [22]:
# results for 1 step
lowest1, lowest_idx1, list_mean1, results_mean1 = get_lowest_error(results1)
lowest1, lowest_idx1

(0.4132182539682539, Index(['c: 1.5; k: 0.4,0.5; xy'], dtype='object'))

In [18]:
# results for 2 steps
list_xy = []
list_xyz = []

for col in results_mean:
    if col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
print('Average xy: ' + str(avg_xy))

for col in results_mean:
    if col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xyz = np.mean(list_xyz)
print('Average xyz: ' + str(avg_xyz))

Average xy: 0.44769645796590246
Average xyz: 0.4530144841269841


In [26]:
# results for 1 step
list_xy1 = []
list_xyz1 = []

for col in results_mean1:
    if col.split(';')[2].strip() == 'xy':
        list_xy1.append(results_mean1.at['mean',col])
        
avg_xy = np.mean(list_xy)
print('Average xy: ' + str(avg_xy))

for col in results_mean1:
    if col.split(';')[2].strip() == 'xyz':
        list_xyz1.append(results_mean1.at['mean',col])
        
avg_xyz = np.mean(list_xyz)
print('Average xyz: ' + str(avg_xyz))

Average xy: 0.44769645796590246
Average xyz: 0.4530144841269841


In [58]:
# compare 1 vs 2 steps
stat, p = wilcoxon(list_xy, list_xy1, zero_method='wilcox')
print('Wilcoxon for xy: T = %.3f, p = %.3f' % (stat, p))

stat, p = wilcoxon(list_xyz, list_xyz1, zero_method='wilcox')
print('Wilcoxon for xyz: T = %.3f, p = %.3f' % (stat, p))

Wilcoxon for xy: T = 1893.500, p = 0.535
Wilcoxon for xyz: T = 1824.000, p = 0.368


In [55]:
# get all mean errors for 1 vs 2 steps
list_means_2steps = list(results_mean.loc['mean',:])
list_means_1step = list(results_mean1.loc['mean',:])

# compare 1 vs 2 steps for all dimensions/parameter combinations
stat, p = wilcoxon(list_means_1step, list_means_2steps, zero_method='wilcox')
print('Wilcoxon: T = %.3f, p = %.3f' % (stat, p))

Wilcoxon: T = 94839.000, p = 0.320


In [19]:
# get data for plot (results for 2 steps)
c = [float(x[3:6]) for x in results_new.columns.tolist()]
k = [float(x[11:14]) for x in results_new.columns.tolist()]
dim = [x.strip() for x in results_new.columns.str.split(';').str[2]]
error = results_mean.loc['mean'].tolist()

In [21]:
# create plot (results for 2 steps)

%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot
img = ax2.scatter(c, k, error, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# set labels
ax2.set_ylabel('parameter k', fontsize=14, labelpad=7)
ax2.set_xlabel('parameter c', fontsize=14, labelpad=7)
ax2.set_zlabel('normalized edit distance', fontsize=14, labelpad=7)
plt.title('Average edit distance (2 steps)', fontsize=16)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks)
plt.show()

In [281]:
# generate rotating image

fig3 = plt.figure(figsize=(12,8))
ax3 = mplot3d.Axes3D(fig3)
ax3.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.set_ylabel('parameter k', fontsize=14, labelpad=7)
ax3.set_xlabel('parameter c', fontsize=14, labelpad=7)
ax3.set_zlabel('normalized edit distance', fontsize=14, labelpad=7)

def init():
    ax3.scatter(c, k, error, alpha=0.5, s=38, c=dim_num, cmap=cmap)
    
    return fig,

def animate(i):
    ax3.view_init(elev=10, azim=i*4)
    return fig,

animated = animation.FuncAnimation(fig3, animate, init_func=init, frames=90, interval=50, blit=True)

In [284]:
# save animated fig

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

fig_name = 'rotate_model_lowesterror_3D'
#animated.save(fig_name+'.gif', writer='imagemagick', fps=1000/50)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=5, bitrate=10000)
animated.save(fig_name+'.mp4', writer=writer)